In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import re

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import fasttext
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
import warnings
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np
 
warnings.filterwarnings(action='ignore')

from code_utils.utils import preprocess,aplatir

In [2]:
df_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_ipcc.jsonl', lines= True).dropna(subset=['year'])

In [3]:
df_ipcc=df_ipcc.dropna(subset=['year'])

In [4]:
df_not_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_not_ipcc.jsonl', lines= True)

In [5]:
df_ipcc=df_ipcc[['year','doi', 'title','topics','locations_names','locations_ids']]

In [6]:
df_not_ipcc=df_not_ipcc[['year','doi', 'title','topics','locations_names','locations_ids']]

In [7]:
df_ipcc=df_ipcc.dropna(subset=['year','doi', 'title','topics','locations_names','locations_ids']).drop_duplicates(subset=['doi'])

In [8]:
df_not_ipcc=df_not_ipcc.dropna(subset=['doi', 'title','topics','locations_names','locations_ids'])

In [9]:
data_counts=pd.Series([int(x) for x in list(df_ipcc.year.dropna())]).value_counts()
data_counts=pd.Series(dict(data_counts)).sort_index()

themes visualisation 

In [ ]:
import matplotlib.pyplot as plt

data_counts=pd.Series([int(x) for x in list(df_ipcc.year.dropna())]).value_counts()
data_counts=pd.Series(dict(data_counts)).sort_index()
data_counts=data_counts[-40:]
plt.figure(figsize=(24, 10))
color_dict = {
'FR': '#BE2125'
}
ax = data_counts.plot(kind='bar', color=[color_dict.get(u, 'grey') for u in data_counts.index], width=0.8)

for i, v in enumerate(data_counts):
    ax.text(i, v + 0.1, f'{v}', ha='center', va='bottom', color='black', size=20)
    
plt.suptitle(f"Distribution temporelle des publications du GIEC (top 40)", size=25)
plt.title('Source : OpenAlex\nTraitement : Science et Ingénierie des Données, SIES', size=15, loc='right')

ax.set_xticklabels(data_counts.index, rotation=45, fontsize=20)
ax.set_yticklabels([f'{tick:.0f}' for tick in ax.get_yticks()], fontsize=20)
ax.set_ylabel(f'nombre de publications', fontsize=15)
ax.set_xlabel(f'années', fontsize=15)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

data_counts=pd.Series(aplatir(list(df_ipcc.topics))).value_counts()
plt.figure(figsize=(14, 12))  
ax = data_counts[:20].plot(kind='barh', color='grey')  

for i, v in enumerate(data_counts[:20]):
    ax.text(v + 0.1, i, f'{v}', ha='left', va='center', color='black', size=15)  

plt.suptitle("Les principaux thèmes des publications du GIEC (top 20)", size=20)
plt.title('Source : OpenAlex \nTraitement : Science et Ingénierie des Données, SIES', size=10, loc='right')

yticklabels = [label[:match_list[0]] + '\n ' + label[match_list[0]:] if (match_list := [match.start() for match in re.finditer('and', label)]) and len(match_list) > 0 else label for label in data_counts.index[:20]]
ax.set_yticklabels(yticklabels, fontsize=15)  
ax.set_xticklabels([f'{tick:.0f}' for tick in ax.get_xticks()], fontsize=12)  
ax.set_xlabel('Nombre de publications', fontsize=15)  
ax.set_ylabel('Thèmes', fontsize=15)  
plt.gca().invert_yaxis()  
plt.show()

In [ ]:
import matplotlib.pyplot as plt

data_counts=df_ipcc.locations_names.value_counts()
plt.figure(figsize=(12, 10))  
ax = data_counts[:20].plot(kind='barh', color='grey')  

for i, v in enumerate(data_counts[:20]):
    ax.text(v + 0.1, i, f'{v}', ha='left', va='center', color='black', size=15)  

plt.suptitle("Les principaux journaux des publications du GIEC (top 20)", size=20)
plt.title('Source : OpenAlex \nTraitement : Science et Ingénierie des Données, SIES', size=10, loc='right')

yticklabels = [label[:match_list[1]] + '\n ' + label[match_list[1]:] if (match_list := [match.start() for match in re.finditer('of', label)]) and len(match_list) > 1 else label for label in data_counts.index[:20]]
ax.set_yticklabels(yticklabels, fontsize=15)  
ax.set_xticklabels([f'{tick:.0f}' for tick in ax.get_xticks()], fontsize=12)  
ax.set_xlabel('Nombre de publications', fontsize=15)  
ax.set_ylabel('Journaux', fontsize=15)  
plt.gca().invert_yaxis()  
plt.show()

building the learning dataset

In [10]:
print(len(df_ipcc),len(df_not_ipcc))

48219 48219


In [11]:
df_ipcc['label']=1
df_not_ipcc['label']=0

In [12]:
df_all=pd.concat([df_ipcc,df_not_ipcc]).reset_index()
del df_all['index']

In [13]:
df_all.loc[:,'doi']=df_all.loc[:,'doi'].apply(lambda x: str(x).replace('https://doi.org/',''))

In [14]:
df_all.loc[:,'title']=df_all.loc[:,'title'].apply(lambda x: str(x).replace('}','').replace('{',''))

In [15]:
df_all.loc[:,'topics']=df_all.loc[:,'topics'].apply(lambda x: ' '.join(x))

In [16]:
df_all

,year,doi,title,topics,locations_names,locations_ids,label
0,2019.0,10.3389/978-2-88963-118-6,OceanObs'19: An Ocean of Opportunity. Volume 1,Marine Biodiversity and Ecosystem Functioning,Frontiers research topics,1664-8714,1
1,1900.0,10.1002/andp.19003081208,"\""Uber die Bedeutung des Wasserdampfes und der...",Stratospheric Chemistry and Climate Change Imp...,Annalen der Physik,0003-3804,1
2,1964.0,10.3402/tellusa.v16i1.8885,The parameters of atmospheric turbidity,Dynamics of Ocean Surface Waves and Wind Inter...,Tellus A,0280-6495,1
3,1929.0,10.1080/20014422.1929.11880498,On the Atmospheric Transmission of Sun Radiati...,Aerosols' Impact on Climate and Hydrological C...,Geografiska Annaler,1651-3215,1
4,2013.0,10.1002/rog.20022,A review of global ocean temperature observati...,Oceanic Modeling and Circulation Studies Globa...,Reviews of Geophysics,1944-9208,1
...,...,...,...,...,...,...,...
96433,1967.0,10.1007/bf02349230,The part played by physicians in the patient's...,Crisis Resolution and Home Treatment Teams in ...,Community mental health journal,0010-3853,0
96434,1966.0,10.1038/210701a0,Delayed Fluorescence of Chlorophyll In Vitro a...,Molecular Mechanisms of Photosynthesis and Pho...,Nature,0028-0836,0
96435,1966.0,10.1507/endocrine1927.42.8_880,子宮筋homogenateにおけるacetylcholineと組織蛋白との結合に及ぼすoxy...,Principles of Stereochemical Structure and Nom...,Nippon Naibunpi Gakkai zasshi/Nihon Naibunpi G...,0029-0661,0
96436,1686.0,10.1098/rstl.1686.0008,A voyage of the Emperour of China into the Eas...,Linguistic Studies in Turkic Languages Silk Ro...,Philosophical transactions of the Royal Societ...,0261-0523,0


In [17]:
df_all.to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_model.json')

In [18]:
df_all.columns

Index(['year', 'doi', 'title', 'topics', 'locations_names', 'locations_ids',
       'label'],
      dtype='object')

using fasttext

In [19]:
df_all['label'] = '__label__' + df_all['label'].astype(str)
df_all['category_description'] = df_all['label']+' '+df_all['title']+' '+df_all['topics']+' '+df_all['locations_names']+' '+df_all['locations_ids']

In [21]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [23]:
df_all['category_description'] = df_all['category_description'].map(preprocess)

In [26]:
train, test = train_test_split(df_all, test_size=0.2, random_state=42)

In [27]:
train.shape, test.shape

((77150, 8), (19288, 8))

In [28]:
train.to_csv("teds.train", columns=["category_description"], index=False, header=False)
test.to_csv("teds.test", columns=["category_description"], index=False, header=False)

In [29]:
model = fasttext.train_supervised(input="teds.train", epoch=25, lr=0.1)
model.test("teds.test")

(19286, 0.9589857928030696, 0.9589857928030696)

In [33]:
model.save_model("fasttext_model_teds.bin")

In [34]:
model = fasttext.load_model("fasttext_model_teds.bin")

In [35]:
test_data = pd.read_csv("teds.test", header=None, names=["text"])

In [37]:
true_labels = []
predicted_labels = []

for line in test_data["text"]:
    label, text = line.split(' ', 1)
    true_labels.append(label)
    text = text.replace('\n', '').strip()
    predicted_label = model.predict(text)[0][0]  
    predicted_labels.append(predicted_label)

In [38]:
conf_matrix = confusion_matrix(true_labels, predicted_labels)

In [39]:
conf_matrix

array([[9223,  371],
       [ 420, 9274]], dtype=int64)

In [40]:
accuracy_score(true_labels, predicted_labels)

0.9589900456242223

In [47]:
recall_score(true_labels, predicted_labels, average="weighted")

0.9589900456242223

In [44]:
f1_score(true_labels, predicted_labels, average="weighted")

0.958990321100486